In [12]:
import os
import requests
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI  # Updated import
from langchain.schema import HumanMessage

In [2]:
from transformers import pipeline

In [13]:
"""def get_trending_stocks():
    url = "https://finance.yahoo.com/trending-tickers"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    stocks = []
    for row in soup.select("table tbody tr"):
        ticker = row.find("td").text
        stocks.append(ticker)

    return stocks[:10]  # Get top 10 trending stocks"""

In [26]:
import requests
from bs4 import BeautifulSoup

def get_trending_stocks():
    url = "https://finance.yahoo.com/trending-tickers"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html5lib")  # Change parser


    stocks = []
    for row in soup.select("tr.simpTblRow"):  # Updated selector for Yahoo Finance
        ticker = row.find("td").text
        stocks.append(ticker)

    return stocks[:10]  # Get top 10 trending stocks

# Test
trending_stocks = get_trending_stocks()
print("Trending Stocks:", trending_stocks)


Trending Stocks: []


In [14]:
# Get News Headlines & Perform Sentiment Analysis
os.environ["OPENAI_API_KEY"] = "---"  
llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

def get_stock_news(stock):
    url = f"https://www.moneycontrol.com/news/tags/{stock}.html" #using moneycontrol to scrape data
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    headlines = [h.text for h in soup.find_all("h2")[:5]]
    return headlines if headlines else ["No recent news"]

def analyze_sentiment(news_headlines):
    prompt = f"Analyze sentiment of these stock market news headlines:\n{news_headlines}. " \
             f"Provide a sentiment score from -1 (very negative) to 1 (very positive)."
    
    response = llm([HumanMessage(content=prompt)])
    return float(response.content)

In [15]:
# Predict Future Prices using LSTM
def predict_stock_price(stock):
    try:
        data = yf.download(stock, period="1y")["Adj Close"]
        if data.isnull().values.any():
            data = data.dropna()
    except:
        print(f" Could not fetch data for {stock}")
        return None

    # Scale data
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data.values.reshape(-1, 1))

    # Prepare training data
    X_train, y_train = [], []
    for i in range(50, len(data_scaled)):
        X_train.append(data_scaled[i-50:i, 0])
        y_train.append(data_scaled[i, 0])
    
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Build LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

    # Predict next 10 days
    future_prices = []
    last_50_days = data_scaled[-50:].reshape(1, 50, 1)

    for _ in range(10):
        predicted_price = model.predict(last_50_days)
        future_prices.append(scaler.inverse_transform(predicted_price)[0, 0])
        last_50_days = np.append(last_50_days[:, 1:, :], predicted_price.reshape(1, 1, 1), axis=1)

    return future_prices

In [16]:
#  Step 4: Generate Investment Insights
def generate_insights(stock, sentiment_score, predicted_prices): #gives investment insights based on the sentiment analysis
    avg_future_price = np.mean(predicted_prices) 
    current_price = yf.download(stock, period="1d")["Adj Close"].values[0]
    
    if sentiment_score > 0.5 and avg_future_price > current_price * 1.05:
        return " Strong Buy"
    elif sentiment_score > 0 and avg_future_price > current_price:
        return "Buy"
    elif sentiment_score < -0.5 and avg_future_price < current_price * 0.95:
        return "Strong Sell"
    else:
        return " Hold"

In [17]:
trending_stocks = get_trending_stocks()

# Sentiment Analysis for Each Stock
stock_sentiments = {stock: analyze_sentiment(get_stock_news(stock)) for stock in trending_stocks}
sorted_stocks = sorted(stock_sentiments.items(), key=lambda x: x[1], reverse=True)
top_5_stocks = sorted_stocks[:5] # gives top 5 stocks sorted by sentiment

stock_predictions = {stock: predict_stock_price(stock) for stock, _ in top_5_stocks} #gives prediction for next 10 days for the stock 

#investment recommendations for the same
investment_recommendations = { 
    stock: generate_insights(stock, sentiment, stock_predictions[stock])
    for stock, sentiment in top_5_stocks
}


In [20]:
trending_stocks

[]